In [112]:
import pickle
import os
import sys
import pandas as pd

import nibabel as nib
import stitchSurfs as stitch
import subprocess
import numpy as np
import projectUtils as prjUtils
import importlib
sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
import gen
import bids_naming as names
importlib.reload(names)

<module 'bids_naming' from '/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/bids_naming.py'>

In [4]:
# equivolumetric surface generation
# options:
# LayNii: Takes cortical ribbon segmentation as input 
#   https://github.com/layerfMRI/LAYNII
# cortPro: takes volumes as input (generates pial, white surfaces)
#   https://github.com/caseypaquola/CortPro
# surface_tools (Kasper Wagstyl): takes pial, white surfaces as input
#   https://github.com/kwagstyl/surface_tools


def runCortPro(study:dict, id:str, ses:str, micro_image_pth:str, dir_surfsIn:str, dir_out:str, n_surfs:int, naming_ptrn:str):
    
    prjUtils.make_dir(dir_out)
    
    fs_dir = "/data/mica1/01_programs/freesurfer-7.3.2" # Path to the FreeSurfer directory [required] (should contain standard license file, 'license.txt')"
    sing_dir  = "/data/mica1/01_programs/micapipe-v0.2.0" # Path to the directory with singularities [required] (must contain micapipe-v0.2.3.simg and, if Freesurfer output is not yet available, fastsurfer_gpu.sif)"
    cortPro_pth = "/host/verges/tank/data/daniel/software/CortPro-main/"

    fs_subjDir = names.get_fsPath(study) # subjects-dir: Path to Freesurfer-style SUBJECTS_DIR [required] (if the directory doesn't contain surfaces for the specified subject, Fastsurfer will be run)
    
    cmd = f"{os.path.join(cortPro_pth, 'microstructure_profiling.sh')} --subject-id {gen.fmt_id_ses(id, ses)}" \
    f" --subjects-dir {fs_subjDir}" \
    f" --micro-image {micro_image_pth}" \
    f" --output-dir {dir_out}" \
    f" --fs-dir {fs_dir}" \
    f" --sing-dir {sing_dir}" \
    f" --num-surfaces {n_surfs}" \
    f" --surface-output {naming_ptrn}" \
    f" --keep-inter-files" # optional

    subprocess.run(cmd, shell=True, capture_output=True, text=True)
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        print(f"Return code: {result.returncode}")
        print(f"STDOUT: {result.stdout}")
        print(f"STDERR: {result.stderr}")
    else:
        print(f"Command executed successfully: {cmd}")
        if result.stdout:
            print(f"STDOUT: {result.stdout}")
    """
    return
    
    # call example
    # TAKES LONG TIME TO RUN
    # works when specify fastsurfer SUBJECTS_DIR
    # adds id_ses file to output dir
    # copies input volumes to output dir
    """
    runCortPro(study = PNI, id="PNC021", ses="01", 
            micro_image_pth = names.get_volPath(study = PNI, id = "PNC021", ses = "01", volName = "T1map")[0],
            dir_surfsIn="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI", 
            dir_out="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/", 
            n_surfs=5, naming_ptrn="stitched")
    """

def equiVolSurfs_LayNII(pial, white, n_surfs, out_prefix, laynii_pth = "/host/verges/tank/data/daniel/software/LayNii_v2.9.0"):
    # LayNii: https://github.com/layerfMRI/LAYNII
    # Citation: Huber, L., Poser, B. A., Bandettini, P. A., Arora, K., Wagstyl, K., Cho, S., Goense, J., Nothnagel, N., Morgan, A. T., van den Hurk, J., Mueller A. K., Reynolds, R. C., Glen, D. R., Goebel, R. W., Gulban, O. F. (2021). LayNii: A software suite for layer-fMRI. NeuroImage, 118091. https://doi.org/10.1016/j.neuroimage.2021.118091
    cmd = ""
    subprocess.run(cmd, shell=True)
    return save_names


def mp_MPC(id:str, ses:str, out_pth:str, bids_dir:str, microStrucImg_pth:str, ):
    
    cmd = "micapipe " \
    f"-sub {gen.fmt_id_ses(id, ses)} " \
    f"-out {out_pth} " \
    f"-bids {bids_dir} " \
    f"-MPC -microstructural_img {microStrucImg_pth}"
    subprocess.run(cmd, shell=True)
    return

def run_surfTools(gray, white, n_surfs, out_prefix, smoothing=None, fs:bool=False):
    # https://github.com/kwagstyl/surface_tools
    # uses different versions of python tools, thus run in own conda env
    
    cmd = "conda run -n legacy_kwagstylSurfTools " \
    "generate_equivolumetric_surfaces " \
        f"{gray} {white} {n_surfs} {out_prefix}"
    if fs:
        cmd += f" --software freesurfer"
    if smoothing is not None:
        cmd += f" --smoothing {smoothing}"
    print(cmd)
    return
    result = subprocess.run(cmd, shell=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        print(f"Return code: {result.returncode}")
        print(f"STDOUT: {result.stdout}")
        print(f"STDERR: {result.stderr}")
    else:
        print(f"Success: {cmd}")
        if result.stdout:
            print(f"STDOUT: {result.stdout}")
    return

PNI = {
    'studyName': 'PNI',
    'studyDescrip': '7T',
    'dir_root': '/data/mica3/BIDS_PNI/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'fastsurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

MICS = {
    'studyName': 'MICs',
    'studyDescrip': '3T',
    'dir_root': '/data/mica3/BIDS_MICs/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'freesurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}



In [5]:
# return the number of vertices from a surface path
def get_nVertices_from_surf(surf_pth:str) -> int:
    surf = nib.load(surf_pth)
    n_vertices = surf.darrays[0].data.shape[0]
    return n_vertices

def removeOutsideTemporal(corticalSurf):
    pass

def name_temporal_cortex_surf(id, ses):
    pass

neo_cortex_areas = ["entorhinal", "parahippocampal", "fusiform"] # check these label naming

In [6]:

orig_outPth = "/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs/orig/" 
fileNames = ["sub-Pilot014_ses-a1_hemi-L_space-nativepro_surf-fsLR-32k_label-pial.surf.gii", 
             "sub-Pilot014_ses-a1_hemi-R_space-nativepro_surf-fsLR-32k_label-pial.surf.gii",
             "sub-Pilot014_ses-a1_hemi-L_space-nativepro_surf-fsLR-32k_label-white.surf.gii",
             "sub-Pilot014_ses-a1_hemi-R_space-nativepro_surf-fsLR-32k_label-white.surf.gii",
             "sub-Pilot014_ses-a1_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii",
             "sub-Pilot014_ses-a1_hemi-R_space-T1w_den-0p5mm_label-hipp_inner.surf.gii",
             "sub-Pilot014_ses-a1_hemi-L_space-T1w_den-0p5mm_label-hipp_outer.surf.gii",
             "sub-Pilot014_ses-a1_hemi-R_space-T1w_den-0p5mm_label-hipp_outer.surf.gii"]
for fileName in fileNames:
    surf_pth = os.path.join(orig_outPth, fileName)
    n_vertices = get_nVertices_from_surf(surf_pth)
    print(f"{fileName}: {n_vertices} vertices")

stitched_outPth = "/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs/"
stitchedSurfs = ["sub-Pilot014_ses-a1_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii",
                 "sub-Pilot014_ses-a1_hemi-L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii",
                 "sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii",
                 "sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii"]

for stitchName in stitchedSurfs:
    surf_pth = os.path.join(stitched_outPth, stitchName)
    n_vertices = get_nVertices_from_surf(surf_pth)
    print(f"{stitchName}: {n_vertices} vertices")



sub-Pilot014_ses-a1_hemi-L_space-nativepro_surf-fsLR-32k_label-pial.surf.gii: 32492 vertices
sub-Pilot014_ses-a1_hemi-R_space-nativepro_surf-fsLR-32k_label-pial.surf.gii: 32492 vertices
sub-Pilot014_ses-a1_hemi-L_space-nativepro_surf-fsLR-32k_label-white.surf.gii: 32492 vertices
sub-Pilot014_ses-a1_hemi-R_space-nativepro_surf-fsLR-32k_label-white.surf.gii: 32492 vertices
sub-Pilot014_ses-a1_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii: 14266 vertices
sub-Pilot014_ses-a1_hemi-R_space-T1w_den-0p5mm_label-hipp_inner.surf.gii: 14266 vertices
sub-Pilot014_ses-a1_hemi-L_space-T1w_den-0p5mm_label-hipp_outer.surf.gii: 14266 vertices
sub-Pilot014_ses-a1_hemi-R_space-T1w_den-0p5mm_label-hipp_outer.surf.gii: 14266 vertices
sub-Pilot014_ses-a1_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii: 37741 vertices
sub-Pilot014_ses-a1_hemi-L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii: 37741 vertices
sub-Pilot014_ses-a1_hem

In [6]:
32492+14266

46758

In [7]:
def print_surf_info(root, file_name):
    surf_pth = os.path.join(root, file_name)
    print(f"{file_name}: {get_nVertices_from_surf(surf_pth)} vertices")
    surf = nib.load(surf_pth)
    str(surf.darrays)
    points = surf.darrays[0].data
    tris = surf.darrays[1].data
    print(f"Points:\n{points[0:3]}")
    print(f"Triangles:\n{tris[0:3]}")
    print("-"*40)


print_surf_info(orig_outPth, fileNames[0])
print_surf_info(stitched_outPth, stitchedSurfs[0])

sub-Pilot014_ses-a1_hemi-L_space-nativepro_surf-fsLR-32k_label-pial.surf.gii: 32492 vertices
Points:
[[ -5.2864585 -35.909584   20.899858 ]
 [-23.838322  -30.016209   46.834885 ]
 [-57.097862   -1.2321265  30.850523 ]]
Triangles:
[[ 68  12   0]
 [180  12  68]
 [180  13  12]]
----------------------------------------
sub-Pilot014_ses-a1_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii: 37741 vertices
Points:
[[ -5.2864585 -35.909584   20.899858 ]
 [-23.838322  -30.016209   46.834885 ]
 [-57.097862   -1.2321265  30.850523 ]]
Triangles:
[[ 31  12   0]
 [107  12  31]
 [107  13  12]]
----------------------------------------


In [21]:
# structure of label files
def print_label_info(label_pth):
    label = nib.load(label_pth)
    print(str(label.darrays))
    labels = label.darrays[0].data
    print(f"Labels data shape: {labels.shape}")
    print(f"Unique labels: {np.unique(labels)}")
    print(f"Label values (first 10): {labels[:10]}")
    print(f"Number of data arrays: {len(label.darrays)}")
    if len(label.darrays) > 0:
        print(f"Number of labels: {len(label.darrays[0].data)}")
    labels = label
    #print(f"Labels:\n{labels[0:10]}")
    print("-"*40)

glsr_lbl_r = "/host/verges/tank/data/daniel/parcellations/glasser/glasser-360_conte69_rh.label.gii"
glsr_lbl_l = "/host/verges/tank/data/daniel/parcellations/glasser/glasser-360_conte69_lh.label.gii"

print_label_info(glsr_lbl_r)
print_label_info(glsr_lbl_l)


[<GiftiDataArray label[32492]>]
Labels data shape: (32492,)
Unique labels: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180]
Label values (first 10): [ 35  39  12  26  13 149  85   0 160 133]
Number of data arrays: 1
Number of labels: 32492
----------------------------------------
[<GiftiDataArray label[32492]>]


In [73]:
importlib.reload(stitch)
glsr_lbl = {
    "parcellationName": "Glasser360",
    "parcName": "glsr",
    "type": "label",
    "pth_label_gii":"/host/verges/tank/data/daniel/parcellations/glasser/glasser-360_conte69_rh.label.gii",
    "pth_csv": "/host/verges/tank/data/daniel/parcellations/glasser/glasser_idx-lbls.csv",
    "csv_idx_label_colNames": ["idx", "LName"],
}

dk25_lbl = {
    "parcellationName": "DeKraker25",
    "parcName": "DK25",
    "type": "label",
    "pth_label_gii": "/host/verges/tank/data/daniel/parcellations/DeKraker25/sub-bigbrain_hemi-R_label-hipp_den-0p5mm_DeKraker25.label.gii",
    "pth_csv": "/host/verges/tank/data/daniel/parcellations/DeKraker25/DK25_details.csv",
    "csv_idx_label_colNames": ["idx", "label"],
}

gi = stitch.stitchLabels(ctx_lbl=glsr_lbl, hipp_lbl=dk25_lbl, 
                          outPth="/host/verges/tank/data/daniel/04_inVivoHistology/code/resources/")


[stitch.resolve_OverlapLblVals] Found 25 overlapping label values for cortical and hippocampal parcelletations.
	Offsetting hippocampal labels by 181.
	Saved merged label CSV to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblValDetails_ctx-glsr_hipp-DK25_05Feb2026-1138.csv
	Saved stitched label Gifti to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026-1138.label.gii


In [76]:
len(gi.darrays[0].data)

37741

In [118]:
importlib.reload(prjUtils)
tmplt_path_root = "/host/verges/tank/data/daniel/04_inVivoHistology/code/resources/"
lbl_gii_pth = os.path.join(tmplt_path_root, "stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026-1138.label.gii")
lbl_csv_pth = os.path.join(tmplt_path_root, "stitch_lblValDetails_ctx-glsr_hipp-DK25_05Feb2026-1138.csv")
mask = prjUtils.make_mask(lbl_tmplt_pth=lbl_gii_pth, csv_pth=lbl_csv_pth, 
                   label_col=['lblSrc', 'LName'], label_vals=[['DeKraker25'], ['Medial_Temporal']], 
                   savePath="/host/verges/tank/data/daniel/04_inVivoHistology/code/resources/", saveName="stitch_ctx-glsr_hipp-DK25_05Feb2026-1138_mesialTemporal_mask")

keep_idx = np.where(mask_data.darrays[0].data == 1)[0]
#print(keep_idx)

csv = pd.read_csv(lbl_csv_pth)
#print(csv.head())
for sidx in keep_idx:
    row = csv[csv['idx'] == sidx]
    if not row.empty:
        print(f"{row['LName'].iloc[0]} ({row['SName'].iloc[0]}), label: {row['label'].iloc[0]}")

Mask saved to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_ctx-glsr_hipp-DK25_05Feb2026-1138_mesialTemporal_mask.gii


In [119]:
mask_data = nib.load(mask)
print(type(mask_data))
n_true = np.sum(mask_data.darrays[0].data == 1)
print(f"Number of True values in mask: {n_true}")

<class 'nibabel.gifti.gifti.GiftiImage'>
Number of True values in mask: 34


In [107]:
output_root = "/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs"
in_name = "sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii"
out_name = "sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched_mesialTemporal_mask.gii"

stitchSurf = nib.load(os.path.join(output_root, in_name))
prjUtils.apply_mask(stitchSurf = stitchSurf, 
                    mask=mask_data, 
                    output_surf_path=os.path.join(output_root,out_name))

AttributeError: module 'nibabel' has no attribute 'GiftiDataArray'

Medial_Temporal (EC), label: Medial_Temporal
Medial_Temporal (PreS), label: Medial_Temporal
Medial_Temporal (H), label: Medial_Temporal
Medial_Temporal (PeEc), label: Medial_Temporal
Medial_Temporal (PHA1), label: Medial_Temporal
Medial_Temporal (PHA3), label: Medial_Temporal
Medial_Temporal (TF), label: Medial_Temporal
Medial_Temporal (PHA2), label: Medial_Temporal
Clear Label (Clear), label: Clear Label
Subiculum (Sub), label: Subiculum
CA1 (CA1), label: CA1
CA2 (CA2), label: CA2
CA3 (CA3), label: CA3
CA4 (CA4), label: CA4
DG (DG), label: DG
SRLM (SRLM), label: SRLM
Cyst (Cyst), label: Cyst
Clear Label (Clear), label: Clear Label
Clear Label (Clear), label: Clear Label
Subiculum (Subiculum), label: Subiculum
CA1 (CA1), label: CA1
CA2 (CA2), label: CA2
CA3 (CA3), label: CA3
CA4 (CA4), label: CA4
DG (DG), label: DG
SRLM (SRLM), label: SRLM
Cyst (Cyst), label: Cyst
Clear Label (Clear), label: Clear Label
Clear Label (Clear), label: Clear Label
Clear Label (Clear), label: Clear Label
CA1

In [10]:
import projectUtils as prjUtils
prjUtils.get_names_stitchSurf(id="PNC021", ses="01", ctx_lbl="pial", ctx_surf="fsLR-32k", hipp_lbl="inner", hipp_surf="den-0p5mm", date=True)

('sub-PNC021_ses-01_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched_09Feb2026-1247.surf.gii',
 'sub-PNC021_ses-01_hemi-R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched_09Feb2026-1247.surf.gii')